<a href="https://colab.research.google.com/github/idityaGE/Generative-AI/blob/main/Master%20Vector%20Database%20for%20LLMs/2_Pinecone_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install All the Required Packages

In [ ]:
!pip install langchain
!pip install pinecone-client
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 6.0 MB/s eta 0:00:00


In [ ]:
!pip install google-generativeai
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 13.1 MB/s eta 0:00:00


In [ ]:
!pip install -U langchain-community langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 55.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.5 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.25
    Uninstalling langchain-core-0.3.25:
      Successfully uninstalled langchain-core-0.3.25
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.12
    Uninstalling langchain-0.3.12:
      Successfully uninstalled langchain-0.3.12


In [ ]:
import os

os.environ['GOOGLE_API_KEY'] = ""

## Import All the Required Libraries

In [ ]:
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_google_genai import GoogleGenerativeAI
from langchain.vectorstores import Pinecone
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_pinecone import PineconeVectorStore
import os

## Load the PDF Files

In [ ]:
!mkdir pdfs

In [ ]:
!gdown 1hPQlXrX8FbaYaLypxTmeVOFNitbBMlEE -O pdfs/yolov7paper.pdf
!gdown 1vILwiv6nS2wI3chxNabMgry3qnV67TxM -O pdfs/rachelgreecv.pdf

Downloading...
From: https://drive.google.com/uc?id=1hPQlXrX8FbaYaLypxTmeVOFNitbBMlEE
To: /content/pdfs/yolov7paper.pdf
100% 2.27M/2.27M [00:00<00:00, 166MB/s]
Downloading...
From: https://drive.google.com/uc?id=1vILwiv6nS2wI3chxNabMgry3qnV67TxM
To: /content/pdfs/rachelgreecv.pdf
100% 271k/271k [00:00<00:00, 86.3MB/s]


## Extract the Text from the PDF's

In [ ]:
loader = PyPDFDirectoryLoader("pdfs")
data = loader.load()

In [ ]:
data

[Document(metadata={'source': 'pdfs/yolov7paper.pdf', 'page': 0}, page_content='YOLOv7: Trainable bag-of-freebies sets new state-of-the-art for real-time object\ndetectors\nChien-Yao Wang1, Alexey Bochkovskiy, and Hong-Yuan Mark Liao1\n1Institute of Information Science, Academia Sinica, Taiwan\nkinyiu@iis.sinica.edu.tw, alexeyab84@gmail.com, and liao@iis.sinica.edu.tw\nAbstract\nYOLOv7 surpasses all known object detectors in both\nspeed and accuracy in the range from 5 FPS to 160 FPS\nand has the highest accuracy 56.8% AP among all known\nreal-time object detectors with 30 FPS or higher on GPU\nV100. YOLOv7-E6 object detector (56 FPS V100, 55.9%\nAP) outperforms both transformer-based detector SWIN-\nL Cascade-Mask R-CNN (9.2 FPS A100, 53.9% AP) by\n509% in speed and 2% in accuracy, and convolutional-\nbased detector ConvNeXt-XL Cascade-Mask R-CNN (8.6\nFPS A100, 55.2% AP) by 551% in speed and 0.7% AP\nin accuracy, as well as YOLOv7 outperforms: YOLOR,\nYOLOX, Scaled-YOLOv4, YOLOv5, DE

## Split the Extracted Data into Text Chunks

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)

In [ ]:
text_chunks = text_splitter.split_documents(data)

In [ ]:
text_chunks[0]

Document(metadata={'source': 'pdfs/yolov7paper.pdf', 'page': 0}, page_content='YOLOv7: Trainable bag-of-freebies sets new state-of-the-art for real-time object\ndetectors\nChien-Yao Wang1, Alexey Bochkovskiy, and Hong-Yuan Mark Liao1\n1Institute of Information Science, Academia Sinica, Taiwan\nkinyiu@iis.sinica.edu.tw, alexeyab84@gmail.com, and liao@iis.sinica.edu.tw\nAbstract\nYOLOv7 surpasses all known object detectors in both\nspeed and accuracy in the range from 5 FPS to 160 FPS\nand has the highest accuracy 56.8% AP among all known')

In [ ]:
len(text_chunks)

168

In [ ]:
text_chunks[2]

Document(metadata={'source': 'pdfs/yolov7paper.pdf', 'page': 0}, page_content='DETR, DINO-5scale-R50, ViT-Adapter-B and many other\nobject detectors in speed and accuracy. Moreover, we train\nYOLOv7 only on MS COCO dataset from scratch without\nusing any other datasets or pre-trained weights. Source\ncode is released in https://github.com/WongKinYiu/yolov7.\n1. Introduction\nReal-time object detection is a very important topic in\ncomputer vision, as it is often a necessary component in\ncomputer vision systems. For example, multi-object track-')

In [ ]:
text_chunks[3]

Document(metadata={'source': 'pdfs/yolov7paper.pdf', 'page': 0}, page_content='ing [94, 93], autonomous driving [40, 18], robotics [35, 58],\nmedical image analysis [34, 46], etc. The computing de-\nvices that execute real-time object detection is usually some\nmobile CPU or GPU, as well as various neural processing\nunits (NPU) developed by major manufacturers. For exam-\nple, the Apple neural engine (Apple), the neural compute\nstick (Intel), Jetson AI edge devices (Nvidia), the edge TPU\n(Google), the neural processing engine (Qualcomm), the AI')

## Downlaod the Embeddings

In [ ]:
embedding = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [ ]:
result = embedding.embed_query("How are you!")
result

[0.016848864033818245,
 -0.023982178419828415,
 -0.02566431649029255,
 -0.016278648748993874,
 0.04114809259772301,
 0.004699599463492632,
 0.005870650988072157,
 -0.01797056943178177,
 -0.0073622711934149265,
 0.008471560664474964,
 0.01614786498248577,
 0.024009227752685547,
 0.0045110913924872875,
 -0.04159996658563614,
 0.0032222666777670383,
 -0.023183930665254593,
 -0.012084429152309895,
 -0.008298139087855816,
 -0.007639026734977961,
 -0.02419697307050228,
 0.012122247368097305,
 0.060329828411340714,
 0.002901292871683836,
 -0.00524657079949975,
 0.0025371431838721037,
 -0.03711733967065811,
 -0.0003088336088694632,
 -0.032901812344789505,
 -0.05879323557019234,
 0.07344481348991394,
 -0.07499648630619049,
 0.007145850453525782,
 -0.044786471873521805,
 -0.010418095625936985,
 0.0431811548769474,
 -0.06883051991462708,
 0.03805170953273773,
 0.02998439408838749,
 -0.02814839966595173,
 0.045941174030303955,
 -0.008327487856149673,
 -0.012221308425068855,
 -0.033937495201826096,

In [ ]:
len(result)  # dimenstion

768

## Initializing the Pinecone

In [ ]:
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY', '')

In [ ]:
from langchain_pinecone import PineconeVectorStore
# from pinecone import Pinecone
# # initialize pinecone
# pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = 'test' # put in the name of your pinecone index here

vectorstore = PineconeVectorStore(index_name=index_name, embedding=embedding, pinecone_api_key=PINECONE_API_KEY)

## Create Embeddings for each of the Text Chunk

In [ ]:
vectorstore.add_documents(text_chunks)

['62d41a31-ac81-4caa-bcff-218b32a37587',
 'a63e9358-2783-447b-8930-36af72c5a1da',
 '464ea003-92f6-4033-880b-2a4744c25283',
 '533516ae-3da9-4029-9383-dc57e17034d0',
 'f4025840-53e6-4309-ac65-4de7ead23f61',
 'ce25020e-90b0-4172-9946-f4221233bf3b',
 '1e825a68-e7e4-4c5e-8339-4636ccce1d8d',
 '559a3a74-e4ff-40cc-8881-0ad6ab2c7b3d',
 '5f3b4e1f-1798-4e48-a8c6-fd27d590b469',
 'f46ace51-3f58-47a7-944f-1f10d3436e32',
 '83d6f2fd-fce5-4b5a-9a5e-c6639f556dd0',
 '13a8ca01-0dce-4400-be9e-43a7acb9a347',
 'ad2537ea-6fdd-44e8-be85-d0295da865f4',
 '7344a8d3-2deb-4f8a-b32a-2c0e96e60352',
 'ff71c966-2fb4-4aa7-8330-30117fda34d6',
 '9256d42f-2be9-4e10-876e-c17b82229afa',
 'cf9bf160-8d98-4ac6-91d1-9c07e1f41e8a',
 'd3ec323a-a268-4401-bc3a-0437b531deeb',
 '589b16f5-3f01-4bc4-a7fb-5ee88932f767',
 '5b22657e-286d-4eb7-9aad-e3b4ffaa4389',
 '530e58c3-8773-45aa-a252-c2c02f08f50a',
 '62345aa4-7a9f-4e05-bfff-4f4dc30f7eb1',
 '1152a675-cc32-4fe8-9524-bb2ad1489600',
 '62cd8f57-8c25-4376-b562-b608d5406f2b',
 '5159deac-1ee1-

## Similarity Search

In [ ]:
query = "YOLOv7 outperforms which models"

In [ ]:
res = vectorstore.similarity_search(query)

for r in res:
  print(r.page_content)

Table 9: More comparison (batch=1, no-TRT, without extra object detection training data)
Model #Param. FLOPs Size FPSV 100 APtest / APval APtest
50 APtest
75
YOLOv7-tiny-SiLU 6.2M 13.8G 640 286 38.7% / 38.7% 56.7% 41.7%
PPYOLOE-S [85] 7.9M 17.4G 640 208 43.1% / 42.7% 60.5% 46.6%
YOLOv7 36.9M 104.7G 640 161 51.4% / 51.2% 69.7% 55.9%
YOLOv5-N (r6.1) [23] 1.9M 4.5G 640 159 - / 28.0% - -
YOLOv5-S (r6.1) [23] 7.2M 16.5G 640 156 - / 37.4% - -
If we compare YOLOv7 with YOLOR using the input
resolution 1280, the inference speed of YOLOv7-W6 is 8
fps faster than that of YOLOR-P6, and the detection rate is
also increased by 1% AP. As for the comparison between
YOLOv7-E6 and YOLOv5-X6 (r6.1), the former has 0.9%
AP gain than the latter, 45% less parameters and 63% less
computation, and the inference speed is increased by 47%.
YOLOv7-D6 has close inference speed to YOLOR-E6, but
improves AP by 0.8%. YOLOv7-E6E has close inference
is 127 fps faster and 10.7% more accurate on AP. In ad-
dition, YOLO

## Creating a LLM Model Wrapper

In [ ]:
llm = GoogleGenerativeAI(model="gemini-pro")

In [ ]:
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=vectorstore.as_retriever())


## Q/A

In [ ]:
query = "YOLOv7 outperforms which models"

In [ ]:
qa.invoke(query)

{'query': 'YOLOv7 outperforms which models',
 'result': 'YOLOv7 outperforms YOLOv5-L (r6.1), YOLOv5-X (r6.1), PPYOLOE-L, YOLOR-P6, YOLOR-E6, and YOLOv5-N (r6.1).'}

In [ ]:
query = "Rachel Green Experience"

In [ ]:
qa.invoke(query)

{'query': 'Rachel Green Experience',
 'result': 'I do not have enough information to answer the question.'}

In [ ]:
import sys

In [ ]:
while True:
  user_input = input(f"Input Prompt: ")
  if user_input == 'exit':
    print('Exiting')
    sys.exit()
  if user_input == '':
    continue
  result = qa({'query': user_input})
  print(f"Answer: {result['result']}")

Input Prompt: what is yolo v7
Answer:  YOLOv7 is a real-time object detector which surpasses all known object detectors in both speed and accuracy. It has the highest accuracy of 56.8% AP among all known detectors and can run from 5 FPS to 160 FPS.
Input Prompt: tell me about Rechel Green
Answer:  Rachel Green is a PhD in English from the University of Illinois at Urbana-Champaign. Her dissertation title was “Down on the Farm: World War One and the Emergence of Literary Modernism in the American South.” She also has a MA in English and was awarded a Summer Research Grant, a Graduate College Conference Travel Grant, Most Outstanding Butler Woman, and an Academic Scholarship. She has published extensively and has given multiple conference presentations.
Input Prompt: exit
Exiting


SystemExit: ignored

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
